#Imports

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import re
import string

In [3]:
train_path = '/content/drive/MyDrive/Datasets/train.csv'
test_path = '/content/drive/MyDrive/Datasets/test.csv'

#Προετοιμασία δεδομένων

In [4]:
def load_data(data_dir, test_dir, batch_size=64):
  features = ['Title', 'Description']
  target = 'Class Index'

  train_df = pd.read_csv(data_dir, header = 0)
  test_df = pd.read_csv(test_dir, header = 0)
  train_df[target] = train_df[target] - 1
  test_df[target] = test_df[target] -1

  # Get size of unique values
  n_labels = len(train_df['Class Index'].unique())

  # Training Dataset
  x_train = train_df.drop(target, axis=1)
  y_train = train_df[target]

  full_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batch_size)
  test_size = int(0.8 * len(full_dataset))
  raw_train_ds = full_dataset.take(test_size)
  raw_val_ds = full_dataset.skip(test_size)

  # Testing Dataset
  x_test = test_df.drop(target, axis=1)
  y_test = test_df[target]
  raw_test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)

  return raw_train_ds,raw_val_ds,raw_test_ds,n_labels

#Υλοποίηση μοντέλου με ενσωματώσεις λέξεων

In [5]:
raw_train_ds, raw_val_ds, raw_test_ds, n_labels = load_data(data_dir = train_path,
                                                            test_dir = test_path,
                                                            batch_size = 32)

In [6]:
# Change input to lowercase, remove punctuation marks and also backslash (“\”)
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  strip = tf.strings.regex_replace(lowercase, '\\', ' ')
  return tf.strings.regex_replace(strip , '[%s]' % re.escape(string.punctuation), '')

In [7]:
def vectorize_text(text,label):
  text = tf.expand_dims(text - 1)
  return custom_standardization(text),label

In [ ]:
# Functional Api model
input = tf.keras.Input(name = 'Title',shape=(None,),batch_size=64)
input_2 = tf.keras.Input(name = 'Description',shape=(None,),batch_size=64)

embedding = tf.keras.layers.Embedding(input_dim = 5000, input_length=250, output_dim=1)(input)

embedding_2 = tf.keras.layers.Embedding(input_dim = 5000, input_length=250, output_dim=1)(input_2)

vectorize_layer = tf.keras.layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=5000,
    pad_to_max_tokens=True,
    output_mode = 'int',
    output_sequence_length = 250)(embedding)

vectorize_layer_2 = tf.keras.layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=5000,
    pad_to_max_tokens=True,
    output_mode = 'int',
    output_sequence_length = 250)(embedding_2)

global_average_pooling1d = tf.keras.layers.GlobalAveragePooling1D()(vectorize_layer)
global_average_pooling1d_2 = tf.keras.layers.GlobalAveragePooling1D()(vectorize_layer_2)

dense = tf.keras.layers.Dense(64, activation='relu')(global_average_pooling1d)
dense_1 = tf.keras.layers.Dense(64, activation='relu')(global_average_pooling1d_2)

concatenate = tf.keras.layers.Concatenate([dense,dense_1])

dropout = tf.keras.layers.Dropout(rate = 0.2,)(concatenate)

dense_2 = tf.keras.layers.Dense(n_labels)(dropout)


model = tf.keras.Model(inputs = [input, input_2],outputs = dense_2)

In [ ]:
# Compile and fit the model
model.compile(loss = tf.keras.losses.CategoricalCrossentropy(),
              optimizer = tf.keras.optimizers.Adam(),
              metrics = ['accuracy'])

history = model.fit(raw_train_ds,
                    epochs = 10,
                    validation_data = raw_val_ds)

In [ ]:
# Evaluate model
print(f" Training :{model.evaluate(raw_train_ds)}")
print(f" Testing  :{model.evaluate(raw_test_ds)}")
print(f" Valid    :{model.evaluate(raw_val_ds)}")